In [99]:
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import roc_auc_score
from clockwork_helperfunc import *
from clockwork_helperfunc import evaluation
import clockwork_helperfunc 
from imp import reload  
reload(clockwork_helperfunc)

#configuration
batch_size = 5
num_epochs = 50
number_units = 7

### Data

In [101]:
training = pd.read_pickle('/Users/leilei/Documents/DS1005/CW/truncate_train_data.p')
val = pd.read_pickle('/Users/leilei/Documents/DS1005/CW/truncate_valid_data.p')

In [102]:
patient_feature_train, label_train, label_time_train = downsample(training, proportional = 2)

In [103]:
sum(label_time_train), sum(label_train*label_time_train)

(523590, 232732)

In [104]:
patient_feature_val = val[0] #before cleaning, original data
label_val = val[1]
label_time_val = val[2]

In [15]:
Counter(label_train), Counter(label_val)

(Counter({0: 3535, 1: 1765}), Counter({0: 4051, 1: 549}))

In [62]:
len(patient_feature_train)

5300

In [49]:
set(label_train[:3535])

{0}

### small_try

In [5]:
idx = np.random.randint(0, len(patient_feature_train),10)
patient_feature_train_sub, label_train_sub, label_time_train_sub = patient_feature_train.copy(), label_train.copy(), label_time_train.copy()
patient_feature_train, label_train, label_time_train = patient_feature_train_sub[idx], label_train_sub[idx], label_time_train_sub[idx]

In [35]:
#crop and make it easy to see the change of weight and other stuff
#y axis is 48 and x aixs is timestep
patient_feature_train = [patient_feature_train[i][:i+1, :5] for i in range(len(patient_feature_train))]

In [77]:
len(patient_feature_train), len(label_train), len(label_time_train)

(10, 10, 10)

In [78]:
label_train

array([0, 0, 1, 0, 1, 0, 0, 1, 1, 0])

In [79]:
sum(label_time_train), sum(label_train*label_time_train)

(727, 321)

In [108]:
def evaluation(data_loader, model, mode = 'novel'):
    '''
    This is the evaluation of patient level prediction.
    '''
    label_list = []
    output_list = []
    model.eval()
    
    for i, (data, label, time_list) in enumerate(data_loader):
        data = Variable(data)
        hidden= model.init_hidden()
        #print(data.size()) #[5, 75, 48]
        #print(time_list) #which is a list
        if mode == 'novel':
            hidden, ori_output = model(data, hidden)
            ##print('the hidden size is ')
            #print(hidden[0].data.size()) #batch_size * hidden dimension
            #output is a list of prediction, everyone is batch * two
            #print('output length' + str(len(output)))
            #print('output single element size' + str(output[0].size()))
            output = [np.argmax(ori_output[time_list[i] - 1][i].data.numpy()) for i in range(data.size()[0])]
            #batch size, each one pick the element
            #each prediction for each batch member
            #print(output[0]) 
        else:
            ori_output = model(data, hidden) #should be [torch.FloatTensor of size seq*batch*2]
            #print(ori_output)
            length, batch_sz = data.size()[1], data.size()[0]
            #print(length, batch_sz)
            output = [np.argmax(ori_output[length*i + time_list[i] - 1].data.numpy()) for i in range(batch_sz)]
            #print(output)
            #for i in range(batch_sz):
                #print(i)
                #np.argmax(ori_output[length*i + time_list[i] - 1].data.numpy())
            #output = np.argmax(last.data.numpy(),axis = 1)
            #print(output)
        #now get a list of hidden and a list of outputs
        #print(output)
        label = label[:,0].numpy()
        #print(label == output)
        #label = label.transpose(0,1).contiguous().view(-1).data.numpy()

        label_list.extend(label)
        output_list.extend(output)
    
    label_list = np.array(label_list)
    output_list = np.array(output_list)
    
    one_idx = np.where(label_list == 1)[0]
    zero_idx = np.where(label_list == 0)[0]
    
    label_ones = label_list[one_idx] #both are array, can select index like this
    label_zeros = label_list[zero_idx]
    pre_ones = output_list[one_idx]#now softmax, turn into class
    pre_zeros = output_list[zero_idx]
        

    acc0 = sum(pre_zeros == label_zeros)/len(pre_zeros)
    acc1 = sum(pre_ones ==label_ones)/len(pre_ones)
    acc =  (sum(pre_ones == label_ones) + sum(pre_zeros == label_zeros))/(len(pre_ones) + len(pre_zeros))
    #print(type(label_ones), type(label_zeros), type(pre_ones),type(pre_zeros))
    auc = roc_auc_score(list(label_ones) + list(label_zeros), list(pre_ones) + list(pre_zeros))
    model.train()
    return acc0, acc1,acc,auc




### Model

In [109]:
### Model
#forward
#cell_class, step
class LSTM_Model(nn.Module):
    def __init__(self, number_units, batch_size, mean = 0, std = 0.1, input_dim = 40):
        super(LSTM_Model, self).__init__()
        '''
        scale: the updating frequency, a list. [1,2,4,8,16,32]
        batch_size: the size of batch
        group_size: the number of nodes in each scale, default is 1.
        mean: the mean of Gaussian distribution for initialize weights for hidden layer
        std: the standard devation of the Gaussian distribution for initialize weights for hidden layer
        input_dim: the feature dimension of each time step
        '''
        self.hidden_dim = number_units
        self.input_dim = input_dim
        self.batch_size = batch_size

        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim)
        self.hidden2tag = nn.Linear(self.hidden_dim, 2)
        
    def forward(self, sequence, hidden):#depends on what passed for model.train(), to be filled)
        '''
        sequence: batch  x timestep x number_feature matrix
        hidden: should be h0
        ''' 
        sequence = sequence.transpose(0,1) #now sequence, batch, num_feature
        #print(sequence.size())
        #print(hidden[0].size())
        lstm_out, hidden = self.lstm(sequence, hidden) #hidden is the last one of sequence
        #print('lstm_size')
        #print(lstm_out.size())
        #print(hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sequence)*self.batch_size, -1))
        #previously last step
        #self.hidden2tag(hidden[0])
        #final_score = F.log_softmax(self.hidden2tag(hidden[0]))
        #print(final_score[0])
        tag_scores = F.log_softmax(tag_space)       
        return tag_scores#, final_score[0]#it is batch * class, the first dimension is 1, so remove it

    def init_hidden(self):
        return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim)),
                Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))


In [ ]:
lstm = LSTM_Model(number_units, batch_size)
loss = torch.nn.CrossEntropyLoss(ignore_index=-1)  
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.005)
accuracy_list = []
train_loader, validation_loader = reload_data(batch_size,patient_feature_train, label_train, label_time_train,patient_feature_val, label_val, label_time_val)
for epoch in range(num_epochs):
    for step, (data, label,label_list) in enumerate(train_loader):        
        data, label = Variable(data), Variable(label)
        lstm.zero_grad()
        hidden= lstm.init_hidden()
        output = lstm(data, hidden)
        
        label = label.transpose(0,1).contiguous().view(-1) 
        #output = torch.stack(output, dim=1).view(-1, 2) 
        #print(output.size())
        #print(label.size())
        lossy = loss(output, label)
        lossy.backward()
        optimizer.step()
        #print('one_step')
        if step % 4 ==0:
            print("Epoch: {}; Loss: {}".format(epoch, lossy.data[0]))
                #print('accuracy_on_training: {}'.format(evaluation(train_loader))) 
            acc0, acc1, val_acc, auc = evaluation(validation_loader, lstm,'lstm')
            print('accuracy_on_validation: {}, the acc for live is {}, the acc for dead is {}'.format(val_acc, acc0, acc1)) 
            print('auc ' +str(auc))
    accuracy_list.append(val_acc)
    if ((epoch > 5) and ((accuracy_list[-1] < (accuracy_list[-2] - 0.01)) or (accuracy_list[-1] < (accuracy_list[-3] - 0.01)))):
        print("early stop, accuracy = ", accuracy_list[-2])
        break            

Epoch: 0; Loss: 0.9413533806800842
accuracy_on_validation: 0.12608695652173912, the acc for live is 0.01883101002690144, the acc for dead is 0.9843444227005871
auc 0.501587716364
Epoch: 0; Loss: 0.7538047432899475
accuracy_on_validation: 0.14934782608695651, the acc for live is 0.048911714355588165, the acc for dead is 0.9530332681017613
auc 0.500972491229
Epoch: 0; Loss: 0.8615680932998657
accuracy_on_validation: 0.19717391304347825, the acc for live is 0.10589386157984837, the acc for dead is 0.9275929549902152
auc 0.516743408285
Epoch: 0; Loss: 0.5953865051269531
accuracy_on_validation: 0.23173913043478261, the acc for live is 0.14697970163854243, the acc for dead is 0.9099804305283757
auc 0.528480066083
Epoch: 0; Loss: 0.7673676609992981
accuracy_on_validation: 0.28304347826086956, the acc for live is 0.2103203717290291, the acc for dead is 0.8649706457925636
auc 0.537645508761
Epoch: 0; Loss: 0.6443922519683838
accuracy_on_validation: 0.31326086956521737, the acc for live is 0.250

In [88]:
### Model
#forward
#cell_class, step
class GRU_Model(nn.Module):
    def __init__(self, number_units, batch_size, mean = 0, std = 0.1, input_dim = 48):
        super(GRU_Model, self).__init__()
        '''
        scale: the updating frequency, a list. [1,2,4,8,16,32]
        batch_size: the size of batch
        group_size: the number of nodes in each scale, default is 1.
        mean: the mean of Gaussian distribution for initialize weights for hidden layer
        std: the standard devation of the Gaussian distribution for initialize weights for hidden layer
        input_dim: the feature dimension of each time step
        '''
        self.hidden_dim = number_units
        self.input_dim = input_dim
        self.batch_size = batch_size

        self.gru = nn.GRU(self.input_dim, self.hidden_dim)
        self.hidden2tag = nn.Linear(self.hidden_dim, 2)
        
    def forward(self, sequence, hidden):#depends on what passed for model.train(), to be filled)
        '''
        sequence: batch  x timestep x number_feature matrix
        hidden: should be h0
        ''' 
        sequence = sequence.transpose(0,1)
        gru_out, hidden = self.gru(sequence, hidden)
        tag_space = self.hidden2tag(gru_out.view(len(sequence)*self.batch_size, -1))
        #self.hidden2tag(hidden[0])
        #final_score = F.log_softmax(self.hidden2tag(hidden[0]))
        tag_scores = F.log_softmax(tag_space)       
        return tag_scores#, final_score[0]
    
    def init_hidden(self):
        return Variable(torch.zeros(1, self.batch_size, self.hidden_dim))

In [12]:
gru = GRU_Model(number_units, batch_size)
loss = torch.nn.CrossEntropyLoss(ignore_index=-1)  
optimizer = torch.optim.Adam(gru.parameters(), lr=0.05)
accuracy_list = []
train_loader, validation_loader = reload_data(batch_size)

for epoch in range(num_epochs):
    for step, (data, label) in enumerate(train_loader):        
        data, label = Variable(data), Variable(label)
        gru.zero_grad()
        hidden= gru.init_hidden()
        output = gru(data, hidden)
        
        label = label.transpose(0,1).contiguous().view(-1) 
        lossy = loss(output, label)
        lossy.backward()
        optimizer.step()
        #print('one_step')
        
    print("Epoch: {}; Loss: {}".format(epoch, lossy.data[0]))
        #print('accuracy_on_training: {}'.format(evaluation(train_loader))) 
    acc0, acc1, val_acc, auc = evaluation(train_loader, gru,'gru')
    print('accuracy_on_validation: {}, the acc for live is {}, the acc for dead is {}'.format(val_acc, acc0, acc1)) 
    accuracy_list.append(val_acc)
    if ((epoch > 5) and ((accuracy_list[-1] < (accuracy_list[-2] - 0.01)) or (accuracy_list[-1] < (accuracy_list[-3] - 0.01)))):
        print("early stop, accuracy = ", accuracy_list[-2])
        break            

Epoch: 0; Loss: 0.6647575497627258
<class 'list'> <class 'list'> <class 'list'> <class 'list'>
accuracy_on_validation: 0.5551138934982652, the acc for live is 0.3667269451784597, the acc for dead is 0.7868698720294042
Epoch: 1; Loss: 0.6899204254150391
<class 'list'> <class 'list'> <class 'list'> <class 'list'>
accuracy_on_validation: 0.5857802836023533, the acc for live is 0.8131206607163002, the acc for dead is 0.3061033100774216
Epoch: 2; Loss: 0.6666716933250427
<class 'list'> <class 'list'> <class 'list'> <class 'list'>
accuracy_on_validation: 0.6146345602655001, the acc for live is 0.7729232115570658, the acc for dead is 0.419905882946646
Epoch: 3; Loss: 0.6320374011993408
<class 'list'> <class 'list'> <class 'list'> <class 'list'>
accuracy_on_validation: 0.5793294614572334, the acc for live is 0.955269697912358, the acc for dead is 0.116843083574093
Epoch: 4; Loss: 0.7337309122085571
<class 'list'> <class 'list'> <class 'list'> <class 'list'>
accuracy_on_validation: 0.5993833911